In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('main.csv')

In [8]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE (%)': mape,
        'R2 Score': r2
    }

In [13]:
total_pred = []
total_test = []

In [14]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(n_steps, len(data)):
        X.append(data[i - n_steps:i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [15]:

# Фильтрация по району (можно обернуть в цикл по всем)
rayon_id = 126
for rayon_id in range(1, 127):
    df_rayon = df[df['Район'] == rayon_id].sort_values('Год')
    prices = df_rayon['Цена'].values.reshape(-1, 1)
    
    # Масштабирование
    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(prices)
    
    # Функция для создания последовательностей
    
    
    # Гиперпараметры
    n_steps = 3
    
    X_all, y_all = create_sequences(scaled_prices, n_steps)
    
    # Train/test split (2 года = 2 строки в конце)
    X_train, X_test = X_all[:-2], X_all[-2:]
    y_train, y_test = y_all[:-2], y_all[-2:]
    
    
    model = Sequential()
    model.add(GRU(32, activation='relu', input_shape=(n_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train, y_train, epochs=200, verbose=0)
    
    y_pred = model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test)
    total_test.append(y_test_inv)
    total_pred.append(y_pred_inv)
    # evaluate_model(y_test_inv, y_pred_inv)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [16]:
evaluate_model(y_test_inv, y_pred_inv)

{'MAE': 14526.413905341848,
 'MSE': 248876270.35192674,
 'RMSE': np.float64(15775.812826980635),
 'MAPE (%)': np.float64(6.1548007082733145),
 'R2 Score': -202.20265688629883}

In [21]:
total_test = np.vstack(total_test).flatten()
total_pred = np.vstack(total_pred).flatten()

In [23]:
evaluate_model(total_test, total_pred)

{'MAE': 13168.271960108292,
 'MSE': 272259152.1713873,
 'RMSE': np.float64(16500.27733619612),
 'MAPE (%)': np.float64(4.614977454292515),
 'R2 Score': 0.9448355507209466}